# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import sys
sys.executable


'/Users/sil/miniconda3/envs/jupyter_env/bin/python'

In [2]:
!pip install scipy


  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/c7/d9/214971dae573bd7e9303b56d2612dae439decbfc0dae0f539a591c0562ce/scipy-1.12.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 13.3 MB/s eta 0:00:0000:0100:01


In [6]:
import sys
sys.version


'3.10.13 (main, Sep 11 2023, 08:39:02) [Clang 14.0.6 ]'

In [7]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [8]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [9]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [10]:
#group by (customerID, ProductName) -->sum(quantity)
df_step1 = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': 'sum'}).reset_index()
df_step1

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [11]:
#pivot table --> df, values, index, columns, --> fill NA 0 -->fill_values 0
df_step2 = df_step1.pivot_table(index='ProductName', columns='CustomerID', values='Quantity', fill_value=0)
df_step2


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [12]:
#pdist, squareform: 1ºhacer el import scipy, 2º df=pdis()--> 3ºsquareform(df)metric('euclidean')
distance = pdist(df_step2.values.T, metric='euclidean')

# Convert the distance to a squareform matrix
similarity_matrix = (squareform(distance))
#similarity_matrix
df_step3 = pd.DataFrame(similarity_matrix, 
                        index=df_step2.columns, 
                        columns=df_step2.columns)
df_step3

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [13]:
#filter df(customerID) --> ordenar(sort)-->select TOP5 (df.head())
df_step4 = df_step3.sort_values(by= ['CustomerID']).head(5)
df_step4

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


In [14]:
customer_id = 33


# Get the similarity scores for the target customer
similarities = df_step3[customer_id]

# Find the top 5 most similar customers
df_step4 = pd.DataFrame(similarities.sort_values()[1:6]).reset_index()
df_step4

,CustomerID,33
0,3535,10.488088
1,264,10.488088
2,3317,10.488088
3,2503,10.630146
4,3305,10.677078


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [15]:
#df(step1)+df(step4--> customerID)}merge join} final df-->customerID...
#step5 = pd.concat([df_step1, df_step4], axis=1)
#step5 

In [16]:
step5 = df_step4.merge(df_step1, on='CustomerID').drop(columns=33)
step5

,CustomerID,ProductName,Quantity
0,3535,Anchovy Paste - 56 G Tube,1
1,3535,Apricots - Dried,1
2,3535,Bacardi Breezer - Tropical,1
3,3535,Bandage - Fexible 1x3,1
4,3535,"Beef - Chuck, Boneless",1
...,...,...,...
265,3305,Wine - Ej Gallo Sierra Valley,2
266,3305,Wine - Pinot Noir Latour,1
267,3305,Wine - Redchard Merritt,1
268,3305,Wine - White Cab Sauv.on,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
#group by()-->sum()rank-->sort,ascending=false
step6 = step5.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending=False)
#step6 = step6.sort_values(ascending=False)
step6


,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [19]:
# # Paso 1: merge el DataFrame de step6 con step2(la matriz de productos del cliente)en el campo "ProductName"
produ_reco = step6.merge(df_step2, on='ProductName').loc[:, ['Quantity',33]]
produ_reco

,Quantity,33
ProductName,,
Butter - Unsalted,3,0
Wine - Ej Gallo Sierra Valley,3,0
Towels - Paper / Kraft,3,1
Soup - Campbells Bean Medley,3,0
Wine - Blue Nun Qualitatswein,3,0
...,...,...
Hinge W Undercut,1,0
Ice Cream Bar - Hageen Daz To,1,0
Jagermeister,1,0


In [20]:
# Paso 2: Filtrar para registros donde el cliente seleccionado no ha comprado el producto
produ_reco = produ_reco[produ_reco[33] == 0].sort_values(by='Quantity', ascending= False)
produ_reco

,Quantity,33
ProductName,,
Butter - Unsalted,3,0
Soup - Campbells Bean Medley,3,0
Wine - Blue Nun Qualitatswein,3,0
Wine - Ej Gallo Sierra Valley,3,0
Beans - Wax,2,0
...,...,...
"Soup - Campbells, Cream Of",1,0
"Soup - Canadian Pea, Dry Mix",1,0
Sponge Cake Mix - Chocolate,1,0


In [21]:
# Paso 3: top 5
step7 = produ_reco.head(5)
step7

,Quantity,33
ProductName,,
Butter - Unsalted,3,0
Soup - Campbells Bean Medley,3,0
Wine - Blue Nun Qualitatswein,3,0
Wine - Ej Gallo Sierra Valley,3,0
Beans - Wax,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [22]:
#recommendations= [{},{},{}]-->for
# Paso 1: Crear un diccionario vacío para almacenar las recomendaciones
recommendations_dict = {}


In [23]:
# Paso 2: Crear una lista de CustomerIDs únicos
unique_customer_id = data['CustomerID'].unique()
#unique_customer_id

In [24]:
# Paso 3: Repita la lista de clientes realizando los pasos del 4 al 7 para cada uno 
# y agregando los resultados de cada iteración al diccionario que creó.

for customer in unique_customer_id:
    #step1
    similar_customer = pd.DataFrame(df_step3[customer].sort_values()[1:6])
    similar_customer = similar_customer.reset_index()
    
 
    #step2
    records_customer = similar_customer.merge(df_step1, on='CustomerID').drop(columns=customer)
    #step5 = similar_customer.merge(step5, on='CustomerID').drop(columns=customer)
    
    #step3
    ranking_customer = records_customer.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending=False)
    #step6 = step5.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending=False)
    
    #step4
    recommended_products_customer = ranking_customer.merge(df_step2, on='ProductName').loc[:, ['Quantity', customer]]
    recommended_products_customer = recommended_products_customer[recommended_products_customer[customer] == 0].sort_values(by='Quantity', ascending=False)
    recommended_products_customer = recommended_products_customer.iloc[0:5]

    #produ_reco = step6.merge(df_step2, on='ProductName').loc[:, ['Quantity',customer]]
    #produ_reco = produ_reco[produ_reco[customer] == 0].sort_values(by='Quantity', ascending= False)
    #produ_reco = produ_reco.head(5)
    #step5
    recommendations_dict[customer] = recommended_products_customer.index.tolist()
recommendations_dict


{61288: ['Jagermeister',
  'Chicken - Soup Base',
  'Lime Cordial - Roses',
  'Macaroons - Two Bite Choc',
  'Flavouring - Orange'],
 77352: ['Wine - Valpolicella Masi',
  'Tahini Paste',
  'Tuna - Salad Premix',
  'Chicken - Soup Base',
  'Isomalt'],
 40094: ['Bread - Italian Corn Meal Poly',
  'Puree - Mocha',
  'Tuna - Salad Premix',
  'Sherry - Dry',
  'Beer - Sleemans Cream Ale'],
 23548: ['Squid - Tubes / Tenticles 10/20',
  'Wanton Wrap',
  'Puree - Mocha',
  'Pernod',
  'Sprouts - Baby Pea Tendrils'],
 78981: ['Vanilla Beans',
  'Lettuce - Frisee',
  'Yogurt - Blueberry, 175 Gr',
  'Cinnamon Buns Sticky',
  'Pop Shoppe Cream Soda'],
 83106: ['Cocoa Butter',
  'Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Cheese - Mix',
  'Sauce - Hollandaise'],
 11253: ['Juice - Lime',
  'Pomello',
  'Tomatoes Tear Drop',
  'Skirt - 29 Foot',
  'Wine - Blue Nun Qualitatswein'],
 35107: ['Bread - French Baquette',
  'Bandage - Flexible Neon',
  'Lentils - Red, Dry',
  'Thermometer 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [28]:
step9 = pd.DataFrame(recommendations_dict)
# Transponer el DataFrame para que los CustomerIDs sean las filas y las recomendaciones sean las columnas
step9_2 =step9.T
step9_2

,0,1,2,3,4
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
23548,Squid - Tubes / Tenticles 10/20,Wanton Wrap,Puree - Mocha,Pernod,Sprouts - Baby Pea Tendrils
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...
49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
41286,Veal - Inside,Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style,Wasabi Powder
85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [29]:
#pdist, squareform: 1ºhacer el import scipy, 2º df=pdis()--> 3ºsquareform(df)metric('euclidean')
distance_2 = pdist(df_step2.values.T, metric='cosine')

# Convert the distance to a squareform matrix
similarity_matrix_2 = (squareform(distance_2))
#similarity_matrix
df_step10 = pd.DataFrame(similarity_matrix_2, 
                        index=df_step2.columns, 
                        columns=df_step2.columns)
df_step10


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,0.886781,0.774648,0.839772,0.843681,0.848243,0.915650,0.884009,0.875835,0.868135,...,0.885703,0.930549,0.895154,0.955144,0.872026,0.946249,0.891093,0.922307,0.888643,0.897020
200,0.886781,0.000000,0.861622,0.868815,0.918555,0.926067,0.795510,0.935250,0.864456,0.846889,...,0.834109,0.875937,0.801003,0.839743,0.885697,0.915985,0.744661,0.861214,0.900539,0.896524
264,0.774648,0.861622,0.000000,0.879829,0.817627,0.903428,0.855401,0.942005,0.862039,0.907695,...,0.828554,0.902768,0.868942,0.880384,0.808039,0.784998,0.823027,0.857563,0.802031,0.897020
356,0.839772,0.868815,0.879829,0.000000,0.901204,0.908448,0.862918,0.848803,0.934606,0.924994,...,0.864555,0.920991,0.875755,0.872427,0.890811,0.885348,0.922566,0.864967,0.835782,0.865764
412,0.843681,0.918555,0.817627,0.901204,0.000000,0.872402,0.843972,0.825672,0.885161,0.841452,...,0.894287,0.768755,0.890907,0.889368,0.846130,0.863288,0.924455,0.856285,0.828344,0.809510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.946249,0.915985,0.784998,0.885348,0.863288,0.842051,0.850545,0.875503,0.868375,0.899353,...,0.877321,0.854225,0.912473,0.843081,0.755807,0.000000,0.779202,0.839396,0.881952,0.926312
98069,0.891093,0.744661,0.823027,0.922566,0.924455,0.919993,0.871885,0.901903,0.879989,0.847056,...,0.889526,0.865745,0.885994,0.927741,0.802092,0.779202,0.000000,0.824781,0.844531,0.863140
98159,0.922307,0.861214,0.857563,0.864967,0.856285,0.873163,0.867060,0.866702,0.835112,0.866645,...,0.842378,0.795676,0.867460,0.917521,0.847048,0.839396,0.824781,0.000000,0.840744,0.881655


In [30]:
cosine_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(df_step2.T, 'cosine'))),
                                index=df_step2.columns,
                                columns=df_step2.columns)

cosine_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [32]:
recommendations_dict_2 = {}
for customer in unique_customer_id:
    #step1
    similar_customer = pd.DataFrame(cosine_dist_norm[customer].sort_values()[1:6])
    similar_customer = similar_customer.reset_index()
    
 
    #step2
    records_customer = similar_customer.merge(df_step1, on='CustomerID').drop(columns=customer)
    #step5 = similar_customer.merge(step5, on='CustomerID').drop(columns=customer)
    
    #step3
    ranking_customer = records_customer.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending=False)
    #step6 = step5.groupby('ProductName').agg({'Quantity': 'sum'}).sort_values('Quantity', ascending=False)
    
    #step4
    recommended_products_customer = ranking_customer.merge(df_step2, on='ProductName').loc[:, ['Quantity', customer]]
    recommended_products_customer = recommended_products_customer[recommended_products_customer[customer] == 0].sort_values(by='Quantity', ascending=False)
    recommended_products_customer = recommended_products_customer.iloc[0:5]

    #produ_reco = step6.merge(df_step2, on='ProductName').loc[:, ['Quantity',customer]]
    #produ_reco = produ_reco[produ_reco[customer] == 0].sort_values(by='Quantity', ascending= False)
    #produ_reco = produ_reco.head(5)
    #step5
    recommendations_dict_2[customer] = recommended_products_customer.index.tolist()
recommendations_dict_2


{61288: ['Shrimp - Baby, Warm Water',
  'Lettuce - Spring Mix',
  'Table Cloth 62x114 White',
  'Initation Crab Meat',
  'Sauce - Demi Glace'],
 77352: ['Turkey - Oven Roast Breast',
  'Oregano - Dry, Rubbed',
  'Tea - Decaf Lipton',
  'Lamb - Ground',
  'Wiberg Super Cure'],
 40094: ['Bread - Italian Corn Meal Poly',
  'Lamb - Pieces, Diced',
  'Durian Fruit',
  'Fish - Scallops, Cold Smoked',
  'Apricots - Halves'],
 23548: ['Muffin - Zero Transfat',
  'Mustard - Seed',
  'Chocolate - Feathers',
  'Cookies - Assorted',
  'Fondant - Icing'],
 78981: ['Vaccum Bag 10x13',
  'Lamb - Whole, Fresh',
  'Rosemary - Primerba, Paste',
  'Cheese Cloth No 100',
  'Wine - Hardys Bankside Shiraz'],
 83106: ['Wine - Ruffino Chianti',
  'Wonton Wrappers',
  'Beer - Labatt Blue',
  'Turnip - White, Organic',
  'Muffin - Carrot Individual Wrap'],
 11253: ['Ecolab - Lime - A - Way 4/4 L',
  'Knife Plastic - White',
  'Liners - Banana, Paper',
  'Pastry - Choclate Baked',
  'Gatorade - Xfactor Berry'],


In [33]:
step11 = pd.DataFrame(recommendations_dict_2)
# Transponer el DataFrame para que los CustomerIDs sean las filas y las recomendaciones sean las columnas
step11_2 =step11.T
step11_2

,0,1,2,3,4
61288,"Shrimp - Baby, Warm Water",Lettuce - Spring Mix,Table Cloth 62x114 White,Initation Crab Meat,Sauce - Demi Glace
77352,Turkey - Oven Roast Breast,"Oregano - Dry, Rubbed",Tea - Decaf Lipton,Lamb - Ground,Wiberg Super Cure
40094,Bread - Italian Corn Meal Poly,"Lamb - Pieces, Diced",Durian Fruit,"Fish - Scallops, Cold Smoked",Apricots - Halves
23548,Muffin - Zero Transfat,Mustard - Seed,Chocolate - Feathers,Cookies - Assorted,Fondant - Icing
78981,Vaccum Bag 10x13,"Lamb - Whole, Fresh","Rosemary - Primerba, Paste",Cheese Cloth No 100,Wine - Hardys Bankside Shiraz
...,...,...,...,...,...
49005,Scallops - 10/20,Wiberg Super Cure,Tilapia - Fillets,Squid - Tubes / Tenticles 10/20,Crab - Imitation Flakes
41286,Scallops - 10/20,Bread - English Muffin,Truffle Cups - Brown,Cattail Hearts,Ezy Change Mophandle
85878,"Lamb - Pieces, Diced",Beer - Rickards Red,"Rosemary - Primerba, Paste",Beer - Original Organic Lager,Mussels - Frozen
68506,"Appetizer - Mini Egg Roll, Shrimp",Pastry - Raisin Muffin - Mini,Spinach - Baby,Water - Spring Water 500ml,"Lentils - Red, Dry"


In [ ]:
# cambia mucho el uso de una metrica a otra